In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install import_ipynb
%cd /content/drive/MyDrive/Colab Notebooks/deeplearning
import import_ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/drive/MyDrive/Colab Notebooks/deeplearning


In [2]:
# 필요한 library imort 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
!pip install torchtext==0.10.0
from torchtext.legacy import data 
from torchtext import datasets
import random
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install torch
import torch
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
train_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/deeplearning/data/unsmile+kocohub_train.xlsx').dropna()
test_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/deeplearning/data/unsmile+kocohub_test.xlsx').dropna()

# column 이름 변경 
# train_data.rename(columns = {'document' : 'document'}, inplace = True)
# test_data.rename(columns = {'document' : 'document'}, inplace = True)

# csv 파일로 저장
train_data.to_csv('/content/train_data.csv', index=False)
test_data.to_csv('/content/test_data.csv', index=False)

In [5]:
# 랜덤 시드 고정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [6]:
KERNEL_SIZE = [3,4,5] # 총 3개의 kernel size 사용( KERNEL_SIZE, embed_dimension)

!pip install konlpy
from konlpy.tag import Okt
okt = Okt()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.5 MB/s 
     |████████████████████████████████| 453 kB 65.2 MB/s 


In [7]:
def tokenizer(text):
  text = text.replace(" ", "space")
  token = okt.morphs(text, stem = True)
  if len(token) < max(KERNEL_SIZE): 
    for i in range(0, max(KERNEL_SIZE)-len(token)):
        token.append('<PAD>') # 커널 사이즈 보다 문장의 길이가 작은 경우 에러 방지
  return token

In [41]:
DOCUMENT = data.Field(tokenize = tokenizer, batch_first = True) # 배치 우선 여부(True일 경우 텐서 크기의 0번째 인덱스는 배치사이즈로 설정)
LABEL = data.LabelField(dtype = torch.float)

# {csv컬럼명 : (데이터 컬럼명, Field이름)} / id는 사용 x
fields = {'document': ('document', DOCUMENT), 'label': ('label', LABEL)}

# 데이터 셋 만들기 (정의한 필드에 기반하여 데이터를 불러옴)
train_data, test_data = data.TabularDataset.splits(path = '/content/', # 데이터 파일 경로 
                                                  train = 'train_data.csv',
                                                  test = 'test_data.csv',
                                                  format = 'csv', # 데이터 파일 형식
                                                  fields = fields) 

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

# 결과 확인
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))
print(vars(train_data[3]))

훈련 샘플의 개수 : 8112
테스트 샘플의 개수 : 2898
{'document': ['개꿀', '빨다', 'space', '평등하다', '<PAD>'], 'label': '1.0'}


In [9]:
import pickle

with open('/content/drive/MyDrive/Colab Notebooks/deeplearning/data/train_data','wb') as output:
  for i in train_data:
    pickle.dump(vars(i), output, pickle.HIGHEST_PROTOCOL)
with open('/content/drive/MyDrive/Colab Notebooks/deeplearning/data/test_data','wb') as output:
  for i in test_data:
    pickle.dump(vars(i), output, pickle.HIGHEST_PROTOCOL)

In [42]:
# 단어 집합 만들기
MAX_VOCAB_SIZE = 25000 # 단어 집합의 최대 크기
DOCUMENT.build_vocab(train_data,
                max_size = MAX_VOCAB_SIZE,
                vectors = 'fasttext.simple.300d', # -> 한글 지원
                unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data, test_data)

print('단어 집합의 크기 : {}'.format(len(DOCUMENT.vocab)))
print(DOCUMENT.vocab.stoi) # 생성된 단어 집합 내 단어 확인
BATCH_SIZE = 128

# BucketIterator : 모든 텍스트 작업을 일괄로 처리하고 단어를 인덱스 숫자로 변환 하는것을 도움 
train_loader, valid_loader, test_loader = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.document), # 길이가 유사한 것을 일괄 처리하고, 패딩을 최소화하기위해 길이로 정렬
    sort_within_batch = True) # 내림차순 정렬

print(len(train_loader))

단어 집합의 크기 : 14219
defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7f46a9aae1d0>>, {'<unk>': 0, '<pad>': 1, 'space': 2, '하다': 3, '이': 4, '.': 5, '들': 6, '가': 7, '?': 8, '에': 9, '은': 10, '도': 11, '는': 12, '보다': 13, '을': 14, '..': 15, '아니다': 16, '...': 17, '<PAD>': 18, '있다': 19, '되다': 20, '없다': 21, '다': 22, '여자': 23, '의': 24, '안': 25, '를': 26, '한': 27, '같다': 28, '로': 29, '이다': 30, '으로': 31, '못': 32, ',': 33, '좋다': 34, '만': 35, '사람': 36, '나': 37, '에서': 38, '왜': 39, 'ㅋㅋ': 40, '그렇다': 41, '남자': 42, '고': 43, '들다': 44, '진짜': 45, '나오다': 46, '거': 47, '저': 48, '하고': 49, '않다': 50, '좀': 51, '게': 52, '그': 53, 'ㅋㅋㅋ': 54, '가다': 55, '년': 56, '아': 57, '~': 58, '더': 59, '적': 60, '잘': 61, '네': 62, '페미': 63, '뭐': 64, '말': 65, '너무': 66, '인': 67, '애': 68, '인데': 69, '많다': 70, '지': 71, '야': 72, '그냥': 73, '니': 74, '받다': 75, '알다': 76, '때': 77, '생각': 78, '!': 79, '것': 80, '내': 81, '라': 82, '면': 83, '먹다': 84, '돈': 85, '이쁘다': 86, '결혼': 87, '오다': 88, '??': 89, '냐': 90, 

In [11]:
# Model 
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_kernels, kernel_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings = vocab_size, # 임베딩을 할 단어들의 개수 (단어 집합의 크기)
                                      embedding_dim = embedding_dim, # 임베딩 할 벡터의 차원 (하이퍼파라미터)
                                      padding_idx = pad_idx) # 패딩을 위한 토큰의 인덱스
        self.convs = nn.ModuleList([nn.Conv2d(in_channels = 1, # input channel수 ( ex RGB 이미지 = 3 )
                                              out_channels = n_kernels, # convolution에 의해 생성될 channel의 수
                                              kernel_size = (ksize, embedding_dim)) # ksize만 변화. embedding_dim은 고정
                                    for ksize in kernel_sizes])
        self.fc = nn.Linear(len(kernel_sizes)*n_kernels, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, document):
        embedded = self.embedding(document)
        embedded = embedded.unsqueeze(1) # 특정 위치에 1인 차원을 추가 <-> squeeze : 1인 차원을 제거
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = self.dropout(torch.cat(pooled, dim = 1))
        res = self.fc(cat)
        return self.fc(cat)

In [12]:
# 모델 선언
INPUT_DIM = len(DOCUMENT.vocab)
EMBEDDING_DIM = 300
N_KERNELS = 100
KERNEL_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = DOCUMENT.vocab.stoi[DOCUMENT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_KERNELS, KERNEL_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

print('모델 파라미터 수 :', sum(param.numel() for param in model.parameters() if param.requires_grad))


모델 파라미터 수 : 4626301


In [13]:
# 사전 훈련된 단어 벡터 불러오기
pretrained_weight = DOCUMENT.vocab.vectors
print(pretrained_weight.shape, model.embedding.weight.data.shape)
print(model.embedding.weight.data.copy_(pretrained_weight))

torch.Size([14219, 300]) torch.Size([14219, 300])
tensor([[ 1.9269,  1.4873,  0.9007,  ..., -2.1268, -0.1341, -1.0408],
        [ 0.7694,  2.5574,  0.5716,  ..., -0.9120,  0.3682,  0.7050],
        [ 0.2673,  0.0777,  0.0187,  ..., -0.0153, -0.2184,  0.0476],
        ...,
        [ 0.3338, -0.6270,  1.4804,  ...,  0.5100,  0.4953,  0.4859],
        [-0.8564, -0.4979, -1.7061,  ...,  0.2398, -0.2520,  1.2816],
        [-0.4738, -0.4040, -0.3054,  ..., -1.4559,  0.8764, -0.2060]])


In [14]:
UNK_IDX = DOCUMENT.vocab.stoi[DOCUMENT.unk_token]

# unk, pad token -> 0 처리 
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [15]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss() # BCELoss + sigmoid

In [16]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds==y).float()
    acc = correct.sum() / len(correct)
    return acc

In [17]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.document).squeeze(1) # output_dim = 1
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [18]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.document).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [19]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [20]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model, 'best_model.pt') # 모델 저장
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc:.4f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc:.4f}')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 01 | Epoch Time: 0m 27s
	Train Loss: 0.476 | Train Acc: 0.8052
	 Val. Loss: 0.329 |  Val. Acc: 0.8744
Epoch: 02 | Epoch Time: 0m 22s
	Train Loss: 0.290 | Train Acc: 0.8863
	 Val. Loss: 0.320 |  Val. Acc: 0.8830
Epoch: 03 | Epoch Time: 0m 21s
	Train Loss: 0.213 | Train Acc: 0.9226
	 Val. Loss: 0.274 |  Val. Acc: 0.8993
Epoch: 04 | Epoch Time: 0m 21s
	Train Loss: 0.145 | Train Acc: 0.9479
	 Val. Loss: 0.271 |  Val. Acc: 0.9026
Epoch: 05 | Epoch Time: 0m 21s
	Train Loss: 0.098 | Train Acc: 0.9672
	 Val. Loss: 0.269 |  Val. Acc: 0.9060


In [21]:
torch.save(model.state_dict(), '/content/best_model.pt')

In [22]:
model.load_state_dict(torch.load('/content/best_model.pt'))

test_loss, test_acc = evaluate(model, test_loader, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc:.4f}')

Test Loss: 0.330 | Test Acc: 0.8841


In [23]:
torch.save(model, "/content/drive/MyDrive/Colab Notebooks/deeplearning/trained_model/CNN")

In [74]:
def predict(model, iterator, criterion):

    predicted = []
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.document).squeeze(1)
            rounded_preds = torch.round(torch.sigmoid(predictions))
            predicted.append(rounded_preds)

        
    return predicted

In [75]:
predict_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/deeplearning/data/Ewha_Presentation.xlsx').dropna()

# csv 파일로 저장
predict_data.to_csv('/content/predict_data.csv', index=False)

In [76]:
# 데이터 셋 만들기 (정의한 필드에 기반하여 데이터를 불러옴)
prediction_data = data.TabularDataset(path = '/content/predict_data.csv', # 데이터 파일 경로 
                                          format = 'csv', # 데이터 파일 형식
                                          fields = fields) 

In [77]:
predict_loader = data.BucketIterator(
    prediction_data,
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.document), # 길이가 유사한 것을 일괄 처리하고, 패딩을 최소화하기위해 길이로 정렬
    sort_within_batch = True) # 내림차순 정렬

In [78]:
model.load_state_dict(torch.load('/content/best_model.pt'))

predicted_list = predict(model, predict_loader, criterion)

print(predicted_list)

[tensor([0., 1., 0., 1., 0., 0., 0., 0., 0., 0.])]


In [79]:
predicted_list[0].tolist()

[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [80]:
predict_data['CNN'] = predicted_list[0].tolist()
predict_data.to_excel("/content/drive/MyDrive/Colab Notebooks/deeplearning/data/Ewha_Presentation.xlsx")